In [ ]:
import numpy as np
import sys
import os
import csv
from PIL import Image
from google.colab import drive
drive.mount('/content/drive/')
os.chdir("/content/drive/My Drive/Toroid Images/")

#here I am trying to convert my .ppm files to .csv
def createFileList(myDir, format='.ppm') :
  fileList = []
  print(myDir)
  for root, dirs, files in os.walk(myDir, topdown=False):
    for name in files:
      if name.endswith(format):
        fullName = os.path.join(root,name)
        fileList.append(fullName)
  return fileList

#uploading my images
myFileList= createFileList('/content/drive/My Drive/Toroid Images/image_to_csv.csv/')

for file in myFileList:
  print(file)
  img_file = Image.open(file)
  width, height = img_file.size
  format = img_file.format
  mode = img_file.mode
  img_grey = img_file.convert('L')

  value = np.asarray(img_grey.getdata(), dtype=np.int).reshape(img_grey.size[1], img_grey.size[0])
  value = value.flatten()
  print(value)
  with open("image_to_csv.csv", 'a') as f:
    writer = csv.writer(f)
    writer.writerow(value)
  
  fMRI_Toroiddis_pic= os.path.join('/content/drive/My Drive/Toroid Images/image_to_csv.csv/')
  train_fMRI_pics = os.listdir(fMRI_Toroiddis_pic)
  #Listing what the file names look like
  print(train_fMRI_pics[:10])
  #Listing how many fMRI images there are total
  print('Total fMRI images:', len(os.listdir(fMRI_Toroiddis_pic)))

#Data preprocessing; normalizing data
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1/255)

train_generator= train_datagen.flow_from_directory(
    '/content/drive/My Drive/Toroid Images/image_to_csv.csv/',
    classes = ['cell,sesnum'],
    target_size=(200,200),
    batch_size=68,
    class_mode='binary')

#Building NN, using model.summary to review code

import tensorflow as tf
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape = (200,200,3)), 
                                    tf.keras.layers.Dense(68, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)])
model.summary()

model.compile(optimizer = tf.optimizers.Adagrad(),
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_generator,
      steps_per_epoch=3,  
      epochs=15,
      verbose=1,
      validation_steps=15)

















Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Toroid Images/image_to_csv.csv/
Found 0 images belonging to 1 classes.
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 120000)            0         
_________________________________________________________________
dense_14 (Dense)             (None, 68)                8160068   
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 69        
Total params: 8,160,137
Trainable params: 8,160,137
Non-trainable params: 0
_________________________________________________________________


ValueError: ignored